In [1]:
import time

import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from semantic_router import Route
load_dotenv()
from semantic_router.routers import SemanticRouter
from semantic_router.encoders import FastEmbedEncoder
from util import set_seed
import dataloader
set_seed(22)

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

c:\Users\risko\Desktop\Prompt-Classification\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-26 22:34:23 - datasets - INFO - config.py:54 - <module>() - PyTorch version 2.7.0 available.


In [2]:
def load_batch_data():
    batch_data = dataloader.get_batch_data()
    return batch_data["prompt"].values.tolist()

In [3]:
batch_data = load_batch_data()
batch_sizes = [1, 32, 64, 128, 256]

domain_data = dataloader.get_domain_data()
domain_data.columns = ["prompt", "dataset", 'label']

ood_data = dataloader.get_ood_data()

train_data = dataloader.get_train_datasets()["train"].to_pandas()
train_data.columns = ["prompt", "dataset", 'label']


In [4]:
def inference_rl(rl : SemanticRouter, input):
    name = rl(input).name
    if name:
        return name
    else:
        return "other"
def inference_rl_batch(rl : SemanticRouter, inputs : list):
    outputs = []
    for i in inputs:
        outputs.append(inference_rl(rl, i))
    return outputs

In [5]:
for model in ["sentence-transformers/all-MiniLM-L6-v2", "BAAI/bge-small-en-v1.5"]:
    embedder = FastEmbedEncoder(score_threshold=0.9)
    embedder.name = model # 
    embedder._initialize_client()

    model = model.replace("/", "_")

    law = Route(
        name="law",
        utterances=train_data[train_data['domain']=='law'].sample(5)['text'].values.tolist(),
    )

    finance = Route(
        name="finance",
        utterances=train_data[train_data['domain']=='finance'].sample(5)['text'].values.tolist(),
    )

    healthcare = Route(
        name="healthcare",
        utterances=train_data[train_data['domain']=='healthcare'].sample(5)['text'].values.tolist(),
    )

    routes = [law,finance, healthcare]

    rl = SemanticRouter(encoder=embedder, routes=routes, auto_sync="local")

    results = []
    for text in tqdm(ood_data['prompt'].values.tolist()):
        result = inference_rl(rl, text)
        results.append(result)
    ood_data['pred'] = results
    ood_data.to_csv(f"data/results/semantic_router_ood_results_{model}_5_t-09.csv")

    results = []
    for text in tqdm(domain_data['prompt'].values.tolist()):
        result = inference_rl(rl, text)
        results.append(result)
    domain_data['pred'] = results
    domain_data.to_csv(f"data/results/semantic_router_domain_results_{model}_5_t-09.csv")

    batch_results = []
    for batch_size in tqdm(batch_sizes):
        batches = [
            batch_data[i : i + batch_size] for i in range(0, len(batch_data), batch_size)
        ]
        for batch in batches:
            
            # Time law predictions
            start_time = time.perf_counter()
            preds = inference_rl_batch(rl, batch)
            time_taken = start_time - time.perf_counter()


            batch_results.append({
                "batch_size": batch_size,
                "time_taken": time_taken,
                "results": preds,
                "model_name": "semantic_router",
                "embedding_model": model,
            })
    batch_results_df = pd.DataFrame(batch_results)
    batch_results_df.to_csv(f"data/results/semantic_router_batch_results_{model}_5_t-90.csv")

ImportError: Please install fastembed to use FastEmbedEncoder. You can install it with: `pip install 'semantic-router[fastembed]'`

In [ ]:
for model in ["sentence-transformers/all-MiniLM-L6-v2", "BAAI/bge-small-en-v1.5"]:
    embedder = FastEmbedEncoder(score_threshold=0.9)
    embedder.name = model # 
    embedder._initialize_client()

    model = model.replace("/", "_")

    law = Route(
        name="law",
        utterances=train_data[train_data['domain']=='law']['text'].values.tolist(),
    )

    finance = Route(
        name="finance",
        utterances=train_data[train_data['domain']=='finance']['text'].values.tolist(),
    )

    healthcare = Route(
        name="healthcare",
        utterances=train_data[train_data['domain']=='healthcare']['text'].values.tolist(),
    )

    routes = [law,finance, healthcare]

    rl = SemanticRouter(encoder=embedder, routes=routes, auto_sync="local")

    results = []
    for text in tqdm(ood_data['prompt'].values.tolist()):
        result = inference_rl(rl, text)
        results.append(result)
    ood_data['pred'] = results
    ood_data.to_csv(f"data/results/semantic_router_ood_results_{model}_all_t-09.csv")

    results = []
    for text in tqdm(domain_data['prompt'].values.tolist()):
        result = inference_rl(rl, text)
        results.append(result)
    domain_data['pred'] = results
    domain_data.to_csv(f"data/results/semantic_router_domain_results_{model}_all_t-09.csv")

    batch_results = []
    for batch_size in tqdm(batch_sizes):
        batches = [
            batch_data[i : i + batch_size] for i in range(0, len(batch_data), batch_size)
        ]
        for batch in batches:
            
            # Time law predictions
            start_time = time.perf_counter()
            preds = inference_rl_batch(rl, batch)
            time_taken = start_time - time.perf_counter()


            batch_results.append({
                "batch_size": batch_size,
                "time_taken": time_taken,
                "results": preds,
                "model_name": "semantic_router",
                "embedding_model": model,
            })
    batch_results_df = pd.DataFrame(batch_results)
    batch_results_df.to_csv(f"data/results/semantic_router_batch_results_{model}_all_t-90.csv")


In [ ]:
for model in ["sentence-transformers/all-MiniLM-L6-v2", "BAAI/bge-small-en-v1.5"]:
    embedder = FastEmbedEncoder(score_threshold=0.5)
    embedder.name = model # 
    embedder._initialize_client()

    model = model.replace("/", "_")

    law = Route(
        name="law",
        utterances=train_data[train_data['domain']=='law'].sample(5)['text'].values.tolist(),
    )

    finance = Route(
        name="finance",
        utterances=train_data[train_data['domain']=='finance'].sample(5)['text'].values.tolist(),
    )

    healthcare = Route(
        name="healthcare",
        utterances=train_data[train_data['domain']=='healthcare'].sample(5)['text'].values.tolist(),
    )

    routes = [law,finance, healthcare]

    rl = SemanticRouter(encoder=embedder, routes=routes, auto_sync="local")

    results = []
    for text in tqdm(ood_data['prompt'].values.tolist()):
        result = inference_rl(rl, text)
        results.append(result)
    ood_data['pred'] = results
    ood_data.to_csv(f"data/results/semantic_router_ood_results_{model}_5_t-09.csv")

    results = []
    for text in tqdm(domain_data['prompt'].values.tolist()):
        result = inference_rl(rl, text)
        results.append(result)
    domain_data['pred'] = results
    domain_data.to_csv(f"data/results/semantic_router_domain_results_{model}_5_t-09.csv")

    batch_results = []
    for batch_size in tqdm(batch_sizes):
        batches = [
            batch_data[i : i + batch_size] for i in range(0, len(batch_data), batch_size)
        ]
        for batch in batches:
            
            # Time law predictions
            start_time = time.perf_counter()
            preds = inference_rl_batch(rl, batch)
            time_taken = start_time - time.perf_counter()


            batch_results.append({
                "batch_size": batch_size,
                "time_taken": time_taken,
                "results": preds,
                "model_name": "semantic_router",
                "embedding_model": model,
            })
    batch_results_df = pd.DataFrame(batch_results)
    batch_results_df.to_csv(f"data/results/semantic_router_batch_results_{model}_5_t-50.csv")

In [ ]:
for model in ["sentence-transformers/all-MiniLM-L6-v2", "BAAI/bge-small-en-v1.5"]:
    embedder = FastEmbedEncoder(score_threshold=0.5)
    embedder.name = model # 
    embedder._initialize_client()

    model = model.replace("/", "_")

    law = Route(
        name="law",
        utterances=train_data[train_data['domain']=='law']['text'].values.tolist(),
    )

    finance = Route(
        name="finance",
        utterances=train_data[train_data['domain']=='finance']['text'].values.tolist(),
    )

    healthcare = Route(
        name="healthcare",
        utterances=train_data[train_data['domain']=='healthcare']['text'].values.tolist(),
    )

    routes = [law,finance, healthcare]

    rl = SemanticRouter(encoder=embedder, routes=routes, auto_sync="local")

    results = []
    for text in tqdm(ood_data['prompt'].values.tolist()):
        result = inference_rl(rl, text)
        results.append(result)
    ood_data['pred'] = results
    ood_data.to_csv(f"data/results/semantic_router_ood_results_{model}_all_t-09.csv")

    results = []
    for text in tqdm(domain_data['prompt'].values.tolist()):
        result = inference_rl(rl, text)
        results.append(result)
    domain_data['pred'] = results
    domain_data.to_csv(f"data/results/semantic_router_domain_results_{model}_all_t-09.csv")

    batch_results = []
    for batch_size in tqdm(batch_sizes):
        batches = [
            batch_data[i : i + batch_size] for i in range(0, len(batch_data), batch_size)
        ]
        for batch in batches:
            
            # Time law predictions
            start_time = time.perf_counter()
            preds = inference_rl_batch(rl, batch)
            time_taken = start_time - time.perf_counter()


            batch_results.append({
                "batch_size": batch_size,
                "time_taken": time_taken,
                "results": preds,
                "model_name": "semantic_router",
                "embedding_model": model,
            })
    batch_results_df = pd.DataFrame(batch_results)
    batch_results_df.to_csv(f"data/results/semantic_router_batch_results_{model}_all_t-50.csv")


